In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Experiments on Vertex AI

### Install Vertex AI SDK for Python and other required packages


In [15]:
! pip3 install --upgrade --quiet --user google-cloud-aiplatform

In [16]:
# @title Authentication to GCP
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [18]:
# @title Set GCP information
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [19]:
# @title Create a bucket.
BUCKET_URI = f"gs://mlops-{PROJECT_ID}-0103"
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-ai-hangsik-0103/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-ai-hangsik-0103' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


In [20]:
# @title Service account
shell_output = ! gcloud projects describe  $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print(f"SERVICE_ACCOUNT: {SERVICE_ACCOUNT}")

SERVICE_ACCOUNT: 721521243942-compute@developer.gserviceaccount.com


In [21]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

No changes made to gs://mlops-ai-hangsik-0103/
No changes made to gs://mlops-ai-hangsik-0103/


In [23]:
# @title Import libraries
import os
import uuid

import google.cloud.aiplatform as aiplatform

In [25]:
# @title Initialize Vertex AI SDK for Python
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## Introduction to Vertex AI Experiments

* Learn more about [Experiments]( https://cloud.google.com/vertex-ai/docs/experiments/intro-vertex-ai-experiments).
* Learn more about [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

* - `aiplatform.init()` - Create an experiment instance
* - `aiplatform.start_run()` - Track a specific run within the experiment.

In [30]:
# @title Create experiment
EXPERIMENT_NAME = f"forus-experiments-{uuid.uuid1()}"

aiplatform.init(experiment=EXPERIMENT_NAME)



In [31]:
# @title Start 1nd run in an experiment
aiplatform.start_run("run-1")

# Log Params
metaparams = {}
metaparams["units"] = 128

aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01

aiplatform.log_params(hyperparams)

# Log metrics
metrics = {}
metrics["test_acc"] = 98.7
metrics["train_acc"] = 99.3
aiplatform.log_metrics(metrics)


INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c-run-1 to Experiment: forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c


In [32]:
# @title Get the experiment results
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)


                                                                     0
experiment_name      forus-experiments-17849178-c96d-11ef-a8b0-0242...
run_name                                                         run-1
run_type                                          system.ExperimentRun
state                                                         COMPLETE
param.batch_size                                                  32.0
param.learning_rate                                               0.01
param.epochs                                                     100.0
param.units                                                      128.0
metric.train_acc                                                  99.3
metric.test_acc                                                   98.7


In [33]:
# @title Start 2nd run in an experiment
aiplatform.start_run("run-2")

metaparams = {}
metaparams["units"] = 256  # changed the value
aiplatform.log_params(metaparams)

hyperparams = {}
hyperparams["epochs"] = 100
hyperparams["batch_size"] = 32
hyperparams["learning_rate"] = 0.01
aiplatform.log_params(hyperparams)

metrics = {}
metrics["test_acc"] = 98.8  # value changed
metrics["train_acc"] = 99.5  # value changed
aiplatform.log_metrics(metrics)

INFO:google.cloud.aiplatform.metadata.experiment_resources:Associating projects/721521243942/locations/us-central1/metadataStores/default/contexts/forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c-run-2 to Experiment: forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c


In [34]:
# @title Comparing runs in the same experiment
aiplatform.end_run()

experiment_df = aiplatform.get_experiment_df()
experiment_df = experiment_df[experiment_df.experiment_name == EXPERIMENT_NAME]
print(experiment_df.T)

                                                                     0  \
experiment_name      forus-experiments-17849178-c96d-11ef-a8b0-0242...   
run_name                                                         run-2   
run_type                                          system.ExperimentRun   
state                                                         COMPLETE   
param.batch_size                                                  32.0   
param.learning_rate                                               0.01   
param.epochs                                                     100.0   
param.units                                                      256.0   
metric.train_acc                                                  99.5   
metric.test_acc                                                   98.8   

                                                                     1  
experiment_name      forus-experiments-17849178-c96d-11ef-a8b0-0242...  
run_name                               

In [35]:
# @title Delete the experiment
exp = aiplatform.Experiment(EXPERIMENT_NAME)
try:
    exp.delete()
except Exception as e:
    print(e)

To delete backing tensorboard run, execute the following:
tensorboard_run_artifact = aiplatform.metadata.artifact.Artifact(artifact_name=f"forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c-run-2-tb-run")
tensorboard_run_resource = aiplatform.TensorboardRun(tensorboard_run_artifact.metadata["resourceName"])
tensorboard_run_resource.delete()
tensorboard_run_artifact.delete()
INFO:google.cloud.aiplatform.base:Deleting Context : projects/721521243942/locations/us-central1/metadataStores/default/contexts/forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c-run-2
INFO:google.cloud.aiplatform.base:Context deleted. . Resource name: projects/721521243942/locations/us-central1/metadataStores/default/contexts/forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c-run-2
INFO:google.cloud.aiplatform.base:Deleting Context resource: projects/721521243942/locations/us-central1/metadataStores/default/contexts/forus-experiments-17849178-c96d-11ef-a8b0-0242ac1c000c-run-2
INFO:google.cloud.aiplatfo

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.

In [ ]:
! rm -rf custom

delete_bucket = False

if delete_bucket:
    ! gsutil rm -rf {BUCKET_URI}